In [1]:
# !pip install scanpy
# !pip install flwr
# !pip install autogluon


## Dataset

In [1]:
import os
os.environ["SCIPY_ARRAY_API"] = "1"

import gdown
import numpy as np
import pandas as pd
import anndata as ad
from sklearn.neural_network import MLPClassifier
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from scipy.sparse import issparse
import matplotlib.pyplot as plt
import seaborn as sns
import random
import torch
import torch.nn as nn
import lightgbm as lgb
import joblib
from sklearn.ensemble import RandomForestClassifier


# Config
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", DEVICE)

os.environ['PYTHONHASHSEED'] = str(SEED)

file_id = "110eYMgseyD32YIS9xOMbOpJ76wnDXahR"
gdown.download(f"https://drive.google.com/uc?id={file_id}", output="TCGA_BRCA_RNA_with_TinX.h5ad", quiet=False)


Using device: cpu


Downloading...
From (original): https://drive.google.com/uc?id=110eYMgseyD32YIS9xOMbOpJ76wnDXahR
From (redirected): https://drive.google.com/uc?id=110eYMgseyD32YIS9xOMbOpJ76wnDXahR&confirm=t&uuid=282f912a-0aaa-41fa-9658-f3457e79f71c
To: /Users/xin/Desktop/DATA5703/Git/TCGA-DNA-RNA-IMAGE-stage-classifier/RNA/Week11/TCGA_BRCA_RNA_with_TinX.h5ad
100%|██████████| 574M/574M [01:26<00:00, 6.63MB/s] 


'TCGA_BRCA_RNA_with_TinX.h5ad'

In [2]:
import scanpy as sc
import pandas as pd

# === Paths ===
adata_path = "TCGA_BRCA_RNA_with_TinX.h5ad"
test_csv_path = "test_metadata_THENEWEST - 28.csv"
train_h5ad_path = "RNA_train.h5ad"
test_h5ad_path = "RNA_test.h5ad"

# === Label mappings ===
label_map = {
    "Stage I": 0,
    "Stage II": 1,
    "Stage III": 2,
    "Stage IV": 3,
}
stage_map = {
    "Stage1": "Stage I",
    "Stage2": "Stage II",
    "Stage3": "Stage III",
    "Stage4": "Stage IV",
}

# === Load .h5ad data ===
adata = sc.read_h5ad(adata_path)
adata.obs["patient_id"] = adata.obs["patient_id"].astype(str)

# === Load test_metadata.csv and fix label format ===
test_df = pd.read_csv(test_csv_path)
test_df["patient_id"] = test_df["patient_id"].astype(str)
test_df["label"] = test_df["label"].str.strip()
test_df["stage"] = test_df["label"].map(stage_map)  # Convert e.g. "Stage4" → "Stage IV"

# === 🔍 Check patient ID consistency ===
csv_patient_ids = set(test_df["patient_id"])
adata_patient_ids = set(adata.obs["patient_id"])
missing_in_adata = csv_patient_ids - adata_patient_ids
if missing_in_adata:
    print("The following patient_id(s) exist in test_metadata.csv but were not found in .h5ad:")
    print(missing_in_adata)
else:
    print("All patient_id(s) in test_metadata.csv are present in the .h5ad dataset.")

# === 1. Extract test set by patient ID ===
test_patients = set(test_df["patient_id"])
is_test = adata.obs["patient_id"].isin(test_patients)
adata_test = adata[is_test].copy()

# De-duplicate: keep only one sample per patient_id
adata_test = adata_test[adata_test.obs.groupby("patient_id").head(1).index]

# Assign correct stage labels from test_metadata.csv
patient_to_stage = dict(zip(test_df["patient_id"], test_df["stage"]))
adata_test.obs["stage"] = adata_test.obs["patient_id"].map(patient_to_stage)

# === 🔍 Check for unmapped test samples ===
unmapped = adata_test.obs[adata_test.obs["stage"].isna()]
if not unmapped.empty:
    print("The following patient_id(s) were found in .h5ad but failed to map a stage label:")
    print(unmapped["patient_id"].tolist())
else:
    print("All test samples successfully mapped to stage labels.")

# === 2. The rest are used as training set ===
adata_train = adata[~is_test].copy()

# === Save output files ===
adata_train.write(train_h5ad_path)
adata_test.write(test_h5ad_path)

# === Final summary ===
print("Training and test sets saved:")
print("Test samples:", adata_test.shape[0], "→", test_h5ad_path)
print("Train samples:", adata_train.shape[0], "→", train_h5ad_path)
print("Test label distribution:")
print(adata_test.obs["stage"].value_counts())

All patient_id(s) in test_metadata.csv are present in the .h5ad dataset.
All test samples successfully mapped to stage labels.


/var/folders/yy/s74n5fc53gv5mw5gdw5l8ch40000gn/T/ipykernel_85757/3291985456.py:54: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_test.obs["stage"] = adata_test.obs["patient_id"].map(patient_to_stage)


Training and test sets saved:
Test samples: 28 → RNA_test.h5ad
Train samples: 1202 → RNA_train.h5ad
Test label distribution:
stage
Stage II     14
Stage III     7
Stage I       6
Stage IV      1
Name: count, dtype: int64


## Train

In [3]:
import numpy as np
import pandas as pd
import anndata as ad
import joblib
from scipy.sparse import issparse
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import SelectKBest, f_classif
from sklearn.metrics import classification_report, confusion_matrix
from imblearn.over_sampling import SMOTE
from autogluon.tabular import TabularDataset, TabularPredictor

# === File paths ===
scaler_path = "RNA_scaler.pkl"
selector_path = "RNA_selector_kbest.pkl"
autogluon_model_path = "autogluon_rna_model"

# === 1. Load data ===
adata = ad.read_h5ad(train_h5ad_path)
X = adata.X.toarray() if issparse(adata.X) else adata.X
y_raw = adata.obs["stage"].values
label_map = {"Stage I": 0, "Stage II": 1, "Stage III": 2, "Stage IV": 3}
label_names = list(label_map.keys())
y = np.array([label_map.get(s, 3) for s in y_raw])  # Default to Stage IV if unknown

# === 2. Sample-level cleaning ===
expr_sum = X.sum(axis=1)
z_scores = (expr_sum - np.mean(expr_sum)) / np.std(expr_sum)
mask = np.abs(z_scores) < 3
X = X[mask]
y = y[mask]

# === 3. Train-validation split ===
X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

# === 4. Scaling ===
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)
joblib.dump(scaler, scaler_path)

# === 5. SelectKBest Feature Selection ===
selector = SelectKBest(score_func=f_classif, k=1000)
X_train_sel = selector.fit_transform(X_train_scaled, y_train)
X_val_sel = selector.transform(X_val_scaled)
joblib.dump(selector, selector_path)

# === 6. SMOTE Over-sampling ===
smote = SMOTE(random_state=42)
X_resampled, y_resampled = smote.fit_resample(X_train_sel, y_train)

# === 7. Prepare for AutoGluon ===
train_df = pd.DataFrame(X_resampled)
train_df["stage"] = y_resampled
val_df = pd.DataFrame(X_val_sel)
val_df["stage"] = y_val

train_data = TabularDataset(train_df)
val_data = TabularDataset(val_df)

# === 8. AutoGluon Training with GPU models and more time ===
predictor = TabularPredictor(
    label="stage",
    path=autogluon_model_path,
    eval_metric="f1_weighted",
    problem_type="multiclass"
)
predictor.fit(
    train_data=train_data,
    tuning_data=val_data,
    use_bag_holdout=True,
    # num_bag_folds=5,
    # num_stack_levels=1,
    time_limit=1800,
    presets="best_quality",
    hyperparameters={
        "GBM": {"ag_args_fit": {"hyperparameter_tune_kwargs": "auto"}},
        "CAT": {"ag_args_fit": {"hyperparameter_tune_kwargs": "auto"}},
        "XGB": {"ag_args_fit": {"hyperparameter_tune_kwargs": "auto"}},
        "RF":  {"ag_args_fit": {"hyperparameter_tune_kwargs": "auto"}}
    }
)

/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:110: UserWarning: Features [17916 18434 19255 20000 20704 21144 22796 22865 23559 23594 25051 25096
 25475 25494 25613 25876 25943 26696 27209 27544 27847 28210 30227 30553
 31799 32852 33003 33969 34230 40615 42909 43522 45975 46464 46819 47370
 47517 49278 49295 49614 49621 49733 50585 52248 52416 52832 53689 53724
 54163 54475 55028 55904] are constant.
  warnings.warn("Features %s are constant." % constant_features_idx, UserWarning)
/opt/anaconda3/lib/python3.12/site-packages/sklearn/feature_selection/_univariate_selection.py:111: RuntimeWarning: invalid value encountered in divide
  f = msb / msw
Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.4.0
Python Version:     3.12.2
Operating System:   Darwin
Platform Machine:   arm64
Platform Version:   Darwin Kernel Version 24.6.0: Mon Jul 14 11:29:54 PDT 2025; root:xnu-11417.140.69~1/RE

In [4]:
# === 9. Evaluation ===
val_preds = predictor.predict(val_data.drop(columns=["stage"]))
print("Validation Classification Report:")
print(classification_report(val_data["stage"], val_preds, target_names=label_names))
print("Confusion Matrix:")
print(pd.DataFrame(confusion_matrix(val_data["stage"], val_preds), index=label_names, columns=label_names))

Validation Classification Report:
              precision    recall  f1-score   support

     Stage I       0.66      1.00      0.80        39
    Stage II       0.83      0.79      0.81       139
   Stage III       0.59      0.54      0.56        54
    Stage IV       0.00      0.00      0.00         9

    accuracy                           0.74       241
   macro avg       0.52      0.58      0.54       241
weighted avg       0.72      0.74      0.72       241

Confusion Matrix:
           Stage I  Stage II  Stage III  Stage IV
Stage I         39         0          0         0
Stage II        13       110         16         0
Stage III        6        19         29         0
Stage IV         1         4          4         0


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])


In [5]:
predictor.leaderboard(val_data, silent=True)

,model,score_test,score_val,eval_metric,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,WeightedEnsemble_L3,0.721473,0.721473,f1_weighted,0.448609,0.657370,1091.608685,0.001092,0.001050,0.080143,3,True,10
1,XGBoost_BAG_L1,0.713832,0.713832,f1_weighted,0.157080,0.175822,113.830715,0.157080,0.175822,113.830715,1,True,4
2,WeightedEnsemble_L2,0.713832,0.713832,f1_weighted,0.158607,0.177003,113.911689,0.001527,0.001181,0.080974,2,True,5
3,LightGBM_BAG_L1,0.709395,0.709395,f1_weighted,0.137733,0.091315,76.668195,0.137733,0.091315,76.668195,1,True,1
4,CatBoost_BAG_L1,0.700685,0.700685,f1_weighted,0.078468,0.329234,897.823853,0.078468,0.329234,897.823853,1,True,3
5,RandomForest_BAG_L2,0.664678,0.664678,f1_weighted,0.447517,0.656320,1091.528543,0.037576,0.025215,1.655122,2,True,7
6,XGBoost_BAG_L2,0.604620,0.604620,f1_weighted,0.556244,0.788320,1163.895713,0.146303,0.157215,74.022292,2,True,9
7,CatBoost_BAG_L2,0.586102,0.586102,f1_weighted,0.511169,0.712035,1582.191760,0.101228,0.080930,492.318339,2,True,8
8,LightGBM_BAG_L2,0.574568,0.574568,f1_weighted,0.490648,0.717419,1174.462964,0.080707,0.086315,84.589543,2,True,6
9,RandomForest_BAG_L1,0.526147,0.526147,f1_weighted,0.036660,0.034733,1.550658,0.036660,0.034733,1.550658,1,True,2


In [6]:
predictor.fit_summary()

*** Summary of fit() ***
Estimated performance of each model:
                 model  score_val  eval_metric  pred_time_val     fit_time  pred_time_val_marginal  fit_time_marginal  stack_level  can_infer  fit_order
0  WeightedEnsemble_L3   0.721473  f1_weighted       0.657370  1091.608685                0.001050           0.080143            3       True         10
1       XGBoost_BAG_L1   0.713832  f1_weighted       0.175822   113.830715                0.175822         113.830715            1       True          4
2  WeightedEnsemble_L2   0.713832  f1_weighted       0.177003   113.911689                0.001181           0.080974            2       True          5
3      LightGBM_BAG_L1   0.709395  f1_weighted       0.091315    76.668195                0.091315          76.668195            1       True          1
4      CatBoost_BAG_L1   0.700685  f1_weighted       0.329234   897.823853                0.329234         897.823853            1       True          3
5  RandomForest_BAG_

{'model_types': {'LightGBM_BAG_L1': 'StackerEnsembleModel_LGB',
  'RandomForest_BAG_L1': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L1': 'StackerEnsembleModel_CatBoost',
  'XGBoost_BAG_L1': 'StackerEnsembleModel_XGBoost',
  'WeightedEnsemble_L2': 'WeightedEnsembleModel',
  'LightGBM_BAG_L2': 'StackerEnsembleModel_LGB',
  'RandomForest_BAG_L2': 'StackerEnsembleModel_RF',
  'CatBoost_BAG_L2': 'StackerEnsembleModel_CatBoost',
  'XGBoost_BAG_L2': 'StackerEnsembleModel_XGBoost',
  'WeightedEnsemble_L3': 'WeightedEnsembleModel'},
 'model_performance': {'LightGBM_BAG_L1': 0.7093947031998783,
  'RandomForest_BAG_L1': 0.5261471983368422,
  'CatBoost_BAG_L1': 0.7006854228665891,
  'XGBoost_BAG_L1': 0.7138316614185216,
  'WeightedEnsemble_L2': 0.7138316614185216,
  'LightGBM_BAG_L2': 0.5745684279509564,
  'RandomForest_BAG_L2': 0.6646779596012864,
  'CatBoost_BAG_L2': 0.5861021964295893,
  'XGBoost_BAG_L2': 0.6046196226392491,
  'WeightedEnsemble_L3': 0.7214732527180796},
 'model_best': 'Weighted

## Client

In [7]:
import numpy as np
import pandas as pd
import anndata as ad
import joblib
import json
from scipy.sparse import issparse
from sklearn.metrics import classification_report, confusion_matrix
import flwr as fl
from autogluon.tabular import TabularPredictor

# ===== Parameter Settings =====
test_h5ad_path   = "RNA_test.h5ad"
scaler_path = "RNA_scaler.pkl"
selector_path = "RNA_selector_kbest.pkl"
autogluon_model_path = "autogluon_rna_model"

SERVER_ADDRESS   = "192.168.0.6:8080"
MODALITY         = "RNA"
WEIGHT           = 0.3

label_map = {"Stage I": 0, "Stage II": 1, "Stage III": 2, "Stage IV": 3}
label_names = list(label_map.keys())
int_to_stage = {v: k for k, v in label_map.items()}

class RNAClient(fl.client.NumPyClient):
    def __init__(self, test_h5ad_path, scaler_path, selector_path, model_path, modality, weight):
        self.modality = modality
        self.weight = weight
        self.rows = []
        self._load_and_predict(test_h5ad_path, scaler_path, selector_path, model_path)

    def _load_and_predict(self, h5ad_path, scaler_path, selector_path, model_path):
        # === 1. Load test data ===
        adata = ad.read_h5ad(h5ad_path)
        X = adata.X.toarray() if issparse(adata.X) else adata.X
        y_raw = adata.obs["stage"].values
        pids = adata.obs["patient_id"].astype(str).values
        y_true = np.array([label_map.get(s, 3) for s in y_raw])

         # === 2. Load scaler & selector ===
        scaler = joblib.load(scaler_path)
        selector = joblib.load(selector_path)

        X_scaled = scaler.transform(X)
        X_selected = selector.transform(X_scaled)

        # === 3. Wrap as DataFrame for AutoGluon ===
        df = pd.DataFrame(X_selected)

        # === 4. Load AutoGluon model and predict ===
        predictor = TabularPredictor.load(model_path)

        y_pred = predictor.predict(df)
        y_prob = predictor.predict_proba(df)

        # AutoGluon returns string labels ("0", "1", ...) → map to int
        y_pred_int = y_pred.astype(int).values

        print("Server_RNA_test Classification Report:")
        print(classification_report(y_true, y_pred_int, target_names=label_names))
        print("Server_RNA_test Confusion Matrix:")
        print(pd.DataFrame(confusion_matrix(y_true, y_pred_int), index=label_names, columns=label_names))

        # === 5. Format to JSON ===
        for i, probs in enumerate(y_prob.values):
            self.rows.append({
                "patient_id": pids[i],
                "probs": probs.tolist(),
                "modality": self.modality,
                "weight": self.weight
            })

        print(f"\n{len(self.rows)} predictions have been generated.")

    def get_parameters(self, config): return []
    def fit(self, parameters, config): return [], 0, {}
    def evaluate(self, parameters, config):
        task = config.get("task", "")
        metrics = {}
        if task == "predict":
            print(f"\n📤 RNA client uploads {len(self.rows)} predictions.")
            metrics = {
                "preds_json": json.dumps(self.rows).encode("utf-8")
            }
        return 0.0, len(self.rows), metrics

# ===== Start client =====
client = RNAClient(test_h5ad_path, scaler_path, selector_path, autogluon_model_path, MODALITY, WEIGHT)

# fl.client.start_numpy_client(server_address=SERVER_ADDRESS, client=client)

Server_RNA_test Classification Report:
              precision    recall  f1-score   support

     Stage I       0.57      0.67      0.62         6
    Stage II       0.71      0.71      0.71        14
   Stage III       0.71      0.71      0.71         7
    Stage IV       0.00      0.00      0.00         1

    accuracy                           0.68        28
   macro avg       0.50      0.52      0.51        28
weighted avg       0.66      0.68      0.67        28

Server_RNA_test Confusion Matrix:
           Stage I  Stage II  Stage III  Stage IV
Stage I          4         2          0         0
Stage II         3        10          1         0
Stage III        0         2          5         0
Stage IV         0         0          1         0

28 predictions have been generated.


/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/opt/anaconda3/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
